In [11]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import protfasta 
import re

from Bio import pairwise2
from Bio.Seq import Seq 
%autoreload 2
%aimport AD_predictor_tools
%aimport AD_comparison_tools
%aimport PlottingTools

# 1. Loading in lists

In [14]:
# Loading in other lists
GSL = pd.read_csv("../output/GSL_with_isoforms.csv")#[["GeneName", "Start", "End", "uniprotID", "Reference", "Sequence"]]
GSL["Reference"] = GSL["Reference"] + ", GSL (" + GSL["uniprotID"] + ", " + GSL["Start"].astype(str) +"_" + GSL["End"].astype(str) + ")"
GSL["TileType"] = "TF"

Soto = pd.read_csv("../data/SotoEtAl_ADs.csv")
Soto["Reference"] = "PMID: " + Soto["Reference (PMID)"] + ", Soto (" + Soto["uniprotID"] + ", " + Soto["Start"].astype(str) +"_" + Soto["End"].astype(str) + ")"
Soto = Soto[["GeneName", "Start", "End", "uniprotID", "Reference", "Sequence"]]
Soto["TileType"] = "TF"

activity_data = pd.read_csv('../data/Staller2021/SupplementalDataSet4_ActivityData_PredictedADs_renorm20210708_uniprotIDs_added.csv', index_col = 0)
activity_data = activity_data[activity_data["RegionType"] == "Prediction"]
active = activity_data[activity_data["Activity_mean"] > 221]
active["Reference"] = "Staller Activity Data (" + activity_data["uniprotID"] + ", " + activity_data["Start"].astype(str) +"_" + activity_data["End"].astype(str) + ")"
active = active[["GeneName", "Start", "End", "uniprotID", "Reference", "ProteinRegionSeq"]]
active = active.rename(columns = {"ProteinRegionSeq" : "Sequence"})
active["GeneName"] = active["GeneName"].str.extract(r'\|.*\|(.*)_')
# Zero-indexing, so adjusting to make one-indexed like other lists
# [Start, End] -> [Start, End)
# It looks like the coordinates for Q8IZM8 that would match the sequence are actually starting at 680
# Based on BLAST and alignment
active["Start"] = active["Start"] + 1
active["End"] = active["End"]
active.at[89, "Start"] = 680
active.at[89, "End"] = 680 + (177-132) - 1
active["TileType"] = "TF"

In [15]:
active["orig_uniprotID"] = active["uniprotID"]
Soto["orig_uniprotID"] = Soto["uniprotID"]

In [16]:
Stanford_isoform_matches_finalized = pd.read_csv("../output/Stanford_isoform_matches_finalized.csv", index_col = 0)
Stanford_isoform_matches_finalized["uniprotID"] = Stanford_isoform_matches_finalized["uniprotID_match"]
Stanford_isoform_matches_finalized = Stanford_isoform_matches_finalized.rename(columns = {"Sublibrary" : "TileType"})
Stanford_isoform_matches_finalized = Stanford_isoform_matches_finalized[["GeneName", "Start", "End", "uniprotID", "Reference", "Sequence", "orig_uniprotID", "Canonical Transcript ID", "TileType"]]
Stanford_isoform_matches_finalized["Reference"] = "DelRosso et al. (" + Stanford_isoform_matches_finalized["uniprotID"] + ", "  + Stanford_isoform_matches_finalized["Start"].astype(str) +"_" + Stanford_isoform_matches_finalized["End"].astype(str) + ")"
Stanford_isoform_matches_finalized["TileType"] = Stanford_isoform_matches_finalized["TileType"].replace({"CRTiles" : "CR",
                                                                                                          "TFTiles" : "TF"})
Stanford_isoform_matches_finalized

,GeneName,Start,End,uniprotID,Reference,Sequence,orig_uniprotID,Canonical Transcript ID,TileType
0,ABRAXAS1,121,200,Q6UWZ7,"DelRosso et al. (Q6UWZ7, 121_200)",LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,Q6UWZ7,ENST00000321945,CR
1,AHR,532,641,P35869,"DelRosso et al. (P35869, 532_641)",QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...,P35869,ENST00000242057,TF
2,AHR,642,721,P35869,"DelRosso et al. (P35869, 642_721)",MKHMQVNGMFENWNSNQFVPFNCPQQDPQQYNVFTDLHGISQEFPY...,P35869,ENST00000242057,TF
3,AKAP8,2,81,O43823,"DelRosso et al. (O43823, 2_81)",DQGYGGYGAWSAGPANTQGAYGTGVASWQGYENYNYYGAQNTSVTT...,O43823,ENST00000269701,TF
4,AKAP8L,2,81,Q9ULX6,"DelRosso et al. (Q9ULX6, 2_81)",SYTGFVQGSETTLQSTYSDTSAQPTCDYGYGTWNSGTNRGYEGYGY...,Q9ULX6,ENST00000397410,TF
...,...,...,...,...,...,...,...,...,...
369,ZFHX4,1732,1811,Q86UP3-5,"DelRosso et al. (Q86UP3-5, 1732_1811)",FQQPQFLFPFYIPGTEFSLGPDLGLPGSATFGMPGMTGMAGSLLED...,Q86UP3,ENST00000651372,TF
370,ZNF200,52,151,P98182,"DelRosso et al. (P98182, 52_151)",TFLPKPSLVQPSQKVKETLVIMKDVSSSLQNRVHPRPLVKLLPKGV...,P98182,ENST00000431561,TF
371,ZNF644,902,981,Q9H582,"DelRosso et al. (Q9H582, 902_981)",PGENATLSYDQNDGFYFEYYEDTGSNNFLHEIHDPQHLETADASLS...,Q9H582,ENST00000337393,TF
372,ZNF687,2,81,Q8N1G0,"DelRosso et al. (Q8N1G0, 2_81)",GDMKTPDFDDLLAAFDIPDIDANEAIHSGPEENEGPGGPGKPEPGV...,Q8N1G0,ENST00000324048,TF


In [19]:
alerasool = pd.read_csv("../data/alerasool_ADs_cleaned.csv", index_col = 0)
alerasool["TileType"] = "TF"
alerasool

,Gene,uniprotID,Start,End,Sequence,Reference,new_AD,TileType
0,NEUROG3,Q9Y4Z2,81,140,RRSRRKKANDRERNRMHNLNSALDALRGVLPTFPDDAKLTKIETLR...,Alerasool et al,False,TF
1,ELF4,Q99607,1,80,MAITLQPSDLIFEFASNGMDDDIHQLEDPSVFPAVIVEQVPYPDLL...,Alerasool et al,False,TF
2,KLF7,O75840,1,80,MDVLASYSIFQELQLVHDTGYFSALPSLEETWQQTCLELERYLQTE...,Alerasool et al,False,TF
3,BRD8,Q9H0E9,368,467,KEECFRSGVAEAPVGSKAPSIDGKEELDLAEKMDIAVSYTGEELDF...,Alerasool et al,False,TF
4,MYCBP,Q99417,1,60,MAHYKAADSKREQFRRYLEKSGVLDTLTKVLVALYEEPEKPNSALD...,Alerasool et al,True,TF
5,C11orf74,Q86VG3-2,1,60,MSAHMSGLEIMDEDQLIKDVLDKFLNCHEQTYDEEFLNTFTHLSQD...,Alerasool et al,True,TF
6,ATOH1,Q92858,1,60,MSRLLHAEEWAEVKELGDHHRQPQPHHLPQPPPPPQPPATLQAREH...,Alerasool et al,True,TF
7,RBPJ,Q06330,256,315,GMALPRLIIRKVDKQTALLDADDPVSQLHKCAFYLKDTERMYLCLS...,Alerasool et al,True,TF
8,DDIT3,P35638,1,80,MAAESLPFSFGTLSSWELEAWYEDLQEVLSSDENGGTYVSPPGNEE...,Alerasool et al,False,TF
9,RELB,Q01201,21,100,RRVARPPAAPELGALGSPDLSSLSLAVSRSTDELEIIDEYIKENGF...,Alerasool et al,False,TF


# 2. Obtaining all isoforms from uniprot

In [25]:
uniprotIDs = set(GSL["uniprotID"]) | set(Soto["uniprotID"]) | set(active["uniprotID"]) | set(Stanford_isoform_matches_finalized["uniprotID"]) | set(alerasool["uniprotID"])
len(uniprotIDs)

608

In [27]:
# saving uniprotIDs to upload
pd.DataFrame(uniprotIDs).to_csv("../data/all_AD_uniprotIDs.csv", header = None, index = None)

In [32]:
#all_isoform_seqs = protfasta.read_fasta("../data/uniprot_known_AD_seqs_isoforms.txt")
all_isoform_seqs = protfasta.read_fasta("../data/uniprot_known_AD_seqs_isoforms_alerasool_added.fasta")
isoform_df = pd.DataFrame({"id":all_isoform_seqs.keys(),
             "seq":all_isoform_seqs.values()})
isoform_df["uniprotID"] = isoform_df["id"].str.split("|").str[1]

# Adding the sequences of A0A669KBM4 and O75152 to isoform_df
# Uniprot says A0A669KBM4 is now obsolete
isoform_df.loc[len(isoform_df.index)] = ['>tr|A0A669KBM4|Release 2021_03/2021_03|02-Jun-2021',
                                         'MAEEQQQPPPQQPDAHQQLPPSAPNSGVALPALVPGLPGTEASALQHKIKNSICKTVQSKVDCILQEVEKFTDLEKLYLYLQLPSGLSNGEKSDQNAMSSSRAQQMHAFSWIRNTLEEHPETSLPKQEVYDEYKSYCDNLGYHPLSAADFGKIMKNVFPNMKARRLGTRGKSKYCYSGLRKKAFVHMPTLPNLDFHKTGDGLEGAEPSGQLQNIDEEVISSACRLVCEWAQKVLSQPFDTVLELARFLVKSHYIGTKSMAALTVMAAAPAGMKGITQPSAFIPTAESNSFQPQVKTLPSPIDAKQQLQRKIQKKQQEQKLQSPLPGESAAKKSESATSNGVTNLPNGNPSILSPQPIGIVVAAVPSPIPVQRTRQLVTSPSPMSSSDGKVLPLNVQVVTQHMQSVKQAPKTPQNVPASPGGDRSARHRYPQILPKPANTSALTIRSPTTVLFTSSPIKTAVVPASHMSSLNVVKMTTISLTPSNSNTPLKHSASVSSATGTTEESRSVPQIKNGSVVSLQSPGSRSSSAGGTSAVEVKVEPETSSDEHPVQCQENSDEAKAPQTPSALLGQKSNTDGALQKPSNEGVIEIKATKVCDQRTKCKSRCNEMLPGTSTGNNQSTITLSVASQNLTFTSSSSPPNGDSINKDPKLCTKSPRKRLSSTLQETQVPPVKKPIVEQLSAATIEGQKQGSVKKDQKVPHSGKTEGSTAGAQIPSKVSVNVSSHIGANQPLNSSALVISDSALEQQTTPSSSPDIKVKLEGSVFLLDSDSKSVGSFNPNGWQQITKDSEFISASCEQQQDISVMTIPEHSDINDLEKSVWELEGMPQDTYSQQLHSQIQESSLNQIQAHSSDQLPLQSELKEFEPSVSQTNESYFPFDDELTQDSIVEELVLMEQQMSMNNSHSYGNCLGMTLQSQSVTPGAPMSSHTSSTHFYHPIHSNGTPIHTPTPTPTPTPTPTPTPTPTSEMIAGSQSLSRESPCSRLAQTTPVDSALGSSRHTPIGTPHSNCSSSVPPSPVECRNPFAFTPISSSMAYHDASIVSSSPVKPMQRPMATHPDKTKLEWMNNGYSGVGNSSVSGHGILPSYQELVEDRFRKPHAFAVPGQSYQSQSRHHDTHFGRLTPVSPVQHQGATVNNTNKQEGFAVPAPLDNKGTNSSASSNFRCRSVSPAVHRQRNLSGSTLYPVSNIPRSNVTPFGSPVTPEVHVFTNVHTDACANNIAQRSQSVPLTVMMQTAFPNALQKQANSKKITNVLLSKLDSDNDDAVRGLGMNNLPSNYTARMNLTQILEPSTVFPSANPQNMIDSSTSVYEFQTPSYLTKSNSTGQINFSPGDNQAQSEIGEQQLDFNSTVKDLLSGDSLQTNQQLVGQGASDLTNTASDFSSDIRLSSELSGSINDLNTLDPNLLFDPGRQQGQDDEATLEELKNDPLFQQICSESMNSMTSSGFEWIESKDHPTVEMLG',
                                         "A0A669KBM4"] 
# O75152
isoform_df.loc[len(isoform_df.index)] = ['>sp|O75152|ZC11A_HUMAN Zinc finger CCCH domain-containing protein 11A OS=Homo sapiens OX=9606 GN=ZC3H11A PE=1 SV=3',
                                         'MPNQGEDCYFFFYSTCTKGDSCPFRHCEAAIGNETVCTLWQEGRCFRQVCRFRHMEIDKKRSEIPCYWENQPTGCQKLNCAFHHNRGRYVDGLFLPPSKTVLPTVPESPEEEVKASQLSVQQNKLSVQSNPSPQLRSVMKVESSENVPSPTHPPVVINAADDDEDDDDQFSEEGDETKTPTLQPTPEVHNGLRVTSVRKPAVNIKQGECLNFGIKTLEEIKSKKMKEKSKKQGEGSSGVSSLLLHPEPVPGPEKENVRTVVRTVTLSTKQGEEPLVRLSLTERLGKRKFSAGGDSDPPLKRSLAQRLGKKVEAPETNIDKTPKKAQVSKSLKERLGMSADPDNEDATDKVNKVGEIHVKTLEEILLERASQKRGELQTKLKTEGPSKTDDSTSGARSSSTIRIKTFSEVLAEKKHRQQEAERQKSKKDTTCIKLKIDSEIKKTVVLPPIVASRGQSEEPAGKTKSMQEVHIKTLEEIKLEKALRVQQSSESSTSSPSQHEATPGARRLLRITKRTGMKEEKNLQEGNEVDSQSSIRTEAKEASGETTGVDITKIQVKRCETMREKHMQKQQEREKSVLTPLRGDVASCNTQVAEKPVLTAVPGITRHLTKRLPTKSSQKVEVETSGIGDSLLNVKCAAQTLEKRGKAKPKVNVKPSVVKVVSSPKLAPKRKAVEMHAAVIAAVKPLSSSSVLQEPPAKKAAVAVVPLVSEDKSVTVPEAENPRDSLVLPPTQSSSDSSPPEVSGPSSSQMSMKTRRLSSASTGKPPLSVEDDFEKLIWEISGGKLEAEIDLDPGKDEDDLLLELSEMIDS',
                                         "O75152"] 
isoform_df

,id,seq,uniprotID
0,tr|A0A024R0Y4|A0A024R0Y4_HUMAN Transcriptional...,MDRLGSFSNDPSDKPPCRGCSSYLMEPYIKCAECGPPPFFLCLQCF...,A0A024R0Y4
1,tr|A0A087WXG3|A0A087WXG3_HUMAN Basic helix-loo...,MSIRPPGEPPSPGGAAMAELKSLSGDAYLALSHGYAAAAAGLAYGA...,A0A087WXG3
2,sp|A1L443|NTM2F_HUMAN NUT family member 2F OS=...,MASNGAYPVLGPGVTVNPGTSLSVFTALPFATPAPGPAHRPPLVTA...,A1L443
3,sp|A6NJG6|ARGFX_HUMAN Arginine-fifty homeobox ...,MRNRMAPENPQPDPFINRNYSNMKVIPPQDPASPSFTLLSKLECSG...,A6NJG6
4,sp|A6NLX3|SPDE4_HUMAN Speedy protein E4 OS=Hom...,MASGQARPPFEEESPQPSTTVRSPEVVVDDEVPGPSAPWIDPSPQP...,A6NLX3
...,...,...,...
1634,sp|Q9Y6Y1-2|CMTA1_HUMAN Isoform 2 of Calmoduli...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1-2
1635,sp|Q9Y6Y1-3|CMTA1_HUMAN Isoform 3 of Calmoduli...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1-3
1636,sp|Q9Y6Y1-4|CMTA1_HUMAN Isoform 4 of Calmoduli...,MWRAEGKWLPKTSRKSVSQSVFCGTSTYCVLNTVPPIEDDHGNSNS...,Q9Y6Y1-4
1637,>tr|A0A669KBM4|Release 2021_03/2021_03|02-Jun-...,MAEEQQQPPPQQPDAHQQLPPSAPNSGVALPALVPGLPGTEASALQ...,A0A669KBM4


# 3. Checking for any canonical isoform matches

In [33]:
# Returns the coordinates of the domain in the canonical isofom, if found
# Must pass in an isoform

def return_canonical_coords(isoform_id, ADs):
    if "-" in isoform_id:
        canonical_isoform_id = isoform_id.split("-")[0]
        full_seq_canonical_seq = isoform_df[isoform_df["uniprotID"] == canonical_isoform_id]["seq"].iloc[0]
        isoform_domain_seq = ADs[ADs["uniprotID"] == isoform_id]["Sequence"].iloc[0]
        match=(re.search(isoform_domain_seq, full_seq_canonical_seq))
        if match:
            start, end = match.span()
            start += 1
            return canonical_isoform_id, start, end
        else:
            print("No canonical match for " + isoform_id)
            print_first_alignment(full_seq_canonical_seq, isoform_domain_seq)
            print()

def print_first_alignment(AA_seq1, AA_seq2):
    seq1 = Seq(AA_seq1) 
    seq2 = Seq(AA_seq2)
    alignments = pairwise2.align.globalxx(seq1, seq2)
    print("\tCanonical full protein sequence:")
    print(alignments[0][0]) 
    print("\tIsoform AD sequence:")
    print(alignments[0][1])

In [36]:
Stanford_output = AD_comparison_tools.return_uniprotID_isoform_mappings(Stanford_isoform_matches_finalized, isoform_df)
active_output = AD_comparison_tools.return_uniprotID_isoform_mappings(active, isoform_df)
Soto_output = AD_comparison_tools.return_uniprotID_isoform_mappings(Soto, isoform_df)
GSL_output = AD_comparison_tools.return_uniprotID_isoform_mappings(GSL, isoform_df)
alerasool_output = AD_comparison_tools.return_uniprotID_isoform_mappings(alerasool, isoform_df)

no matching isoforms for:
Q6SJ96
Tested:
1171    Q6SJ96
Name: uniprotID, dtype: object
2
81
expected:
Q6SJ96
ASAPWPERVPRLLAPRLPSYPPPPPTVGLRSMEQEETYLELYLDQCAAQDGLAPPRSPLFSPVVPYDMYILNASNPDTAF
observed:
['EQEETYLELYLDQCAAQDGLAPPRSPLFSPVVPYDMYILNASNPDTAFNSNPEVKETSGDFSSVDLSFLPDEVTQENKDQ']

no matching isoforms for:
Q6SJ96
Tested:
1171    Q6SJ96
Name: uniprotID, dtype: object
22
101
expected:
Q6SJ96
PPPPPTVGLRSMEQEETYLELYLDQCAAQDGLAPPRSPLFSPVVPYDMYILNASNPDTAFNSNPEVKETSGDFSSVDLSF
observed:
['APPRSPLFSPVVPYDMYILNASNPDTAFNSNPEVKETSGDFSSVDLSFLPDEVTQENKDQPVISKHETEENSESQSPQSR']

no matching isoforms for:
H3BS19
Tested:
Series([], Name: uniprotID, dtype: object)
792
891
expected:
H3BS19
SHAKTFLLAGDAQAEGKDDPLRTGFLPSLAATPFPLPASDLDMEDDAKLDSLITEALNGMEYQSDNPEIDSSFIDVFADEEPSGPRGPSSGHPLKSKAGV
observed:
[]

no matching isoforms for:
H3BS19
Tested:
Series([], Name: uniprotID, dtype: object)
832
911
expected:
H3BS19
LDMEDDAKLDSLITEALNGMEYQSDNPEIDSSFIDVFADEEPSGPRGPSSGHPLKSKAGVTPESKAPPPLPAATPDPQTP
observed:
[]

In [45]:
Soto_output[[len(_) == 0 for _ in Soto_output["matching_isoforms"]]]

,GeneName,Start,End,uniprotID,Reference,Sequence,TileType,orig_uniprotID,matching_isoforms
165,HNF1A,281,631,P20823,"PMID: 1656070, 8288579, Soto (P20823, 281_631)",LAMDTYSGPPPGPGPGPALPAHSSPGLPPPALSPSKVHGVRYGQPA...,TF,P20823,[]


In [37]:
# Choosing first of all matches by default
active_output["uniprotID"] = [_[0] for _ in active_output["matching_isoforms"]]
Soto_output["uniprotID"] = [_[0] for _ in Soto_output["matching_isoforms"]]
GSL_output["uniprotID"] = [_[0] for _ in GSL_output["matching_isoforms"]]

IndexError: list index out of range

In [187]:
# Looking more closely at matches being selected
active_output["num_matches"] = [len(_) for _ in active_output["matching_isoforms"]]
active_output[active_output["num_matches"] > 1]

,GeneName,Start,End,uniprotID,Reference,Sequence,TileType,orig_uniprotID,matching_isoforms,num_matches
17,ZN644,852.0,893.0,Q9H582,"Staller Activity Data (Q9H582, 851.0_893.0)",SYETEDESSWDNVELGDYTTQAIEDETYSDINQEHVNLFPLF,TF,Q9H582,"[Q9H582, Q9H582-2]",2
54,TIGD7,408.0,446.0,Q6NT04,"Staller Activity Data (Q6NT04, 407.0_446.0)",PEYDFQGLEHGDYREILEKCGELETKLDDDRVWLNGDEE,TF,Q6NT04,"[Q6NT04, Q6NT04-2]",2
74,SRBP2,2.0,56.0,Q12772,"Staller Activity Data (Q12772, 1.0_56.0)",DDSGELGGLETMETLTELGDELTLGDIDEMLQFVSNQVGEFPDLFS...,TF,Q12772,"[Q12772, Q12772-2]",2
75,CR3L1,12.0,86.0,Q96BA8,"Staller Activity Data (Q96BA8, 11.0_86.0)",RLFPGSSFLDLGDLNESDFLNNAHFPEHLDHFTENMEDFSNDLFSS...,TF,Q96BA8,"[Q96BA8, Q96BA8-2]",2
81,C2D1A,22.0,60.0,Q6P1N0,"Staller Activity Data (Q6P1N0, 21.0_60.0)",GLLVDLSPDGLMIPEDGANDEELEAEFLALVGGQPPALE,TF,Q6P1N0,"[Q6P1N0, Q6P1N0-2]",2
99,ZFHX4,1405.0,1443.0,Q86UP3,"Staller Activity Data (Q86UP3, 1404.0_1443.0)",ELSEAELQQLYASLPVNGELWAESETMSQDDHGLEQEME,TF,Q86UP3,"[Q86UP3, Q86UP3-4]",2
112,HIF1A,534.0,572.0,Q16665,"Staller Activity Data (Q16665, 533.0_572.0)",ELVEKLFAEDTEAKNPFSTQDTDLDLEMLAPYIPMDDDF,TF,Q16665,"[Q16665, Q16665-2]",2
118,SRBP1,3.0,53.0,P36956,"Staller Activity Data (P36956, 2.0_53.0)",EPPFSEAALEQALGEPCDLDAALLTDIEDMLQLINNQDSDFPGLFD...,TF,P36956,"[P36956, P36956-2, P36956-5]",3
141,CREB3,2.0,55.0,O43889,"Staller Activity Data (O43889, 1.0_55.0)",ELELDAGDQDLLAFLLEESGDLGTAPDEAVRAPLDWALPLSEVPSD...,TF,O43889,"[O43889, O43889-3]",2
143,ETV1,37.0,83.0,P50549,"Staller Activity Data (P50549, 36.0_83.0)",RDLAHDSEELFQDLSQLQETWLAEAQVPDNDEQFVPDYQAESLAFHG,TF,P50549,"[P50549, P50549-3]",2


In [188]:
Soto_output["num_matches"] = [len(_) for _ in Soto_output["matching_isoforms"]]
Soto_output["first_match"] = [_[0] for _ in Soto_output["matching_isoforms"]]
Soto_output["first_match_not_canonical"] = ["-" in _ for _ in Soto_output["first_match"]]
Soto_output[Soto_output["first_match_not_canonical"] & (Soto_output["num_matches"] > 1)]

,GeneName,Start,End,uniprotID,Reference,Sequence,TileType,orig_uniprotID,matching_isoforms,num_matches,first_match,first_match_not_canonical
72,ERG,125,209,P11308-1,"PMID: 14603248, Soto (P11308, 125_209)",MTTNERRVIVPADPTLWSTDHVRQWLEWAVKEYGLPDVNILLFQNI...,TF,P11308,"[P11308-1, P11308-3, P11308-5, P11308-6]",4,P11308-1,True


In [189]:
Soto_output[Soto_output["GeneName"] == "ERG"]

,GeneName,Start,End,uniprotID,Reference,Sequence,TileType,orig_uniprotID,matching_isoforms,num_matches,first_match,first_match_not_canonical
72,ERG,125,209,P11308-1,"PMID: 14603248, Soto (P11308, 125_209)",MTTNERRVIVPADPTLWSTDHVRQWLEWAVKEYGLPDVNILLFQNI...,TF,P11308,"[P11308-1, P11308-3, P11308-5, P11308-6]",4,P11308-1,True
73,ERG,440,486,P11308-3,"PMID: 9681824, Soto (P11308, 440_486)",PHPPALPVTSSSFFAAPNPYWNSPTGGIYPNTRLPTSHMPSHLGTYY,TF,P11308,[P11308-3],1,P11308-3,True


In [190]:
# ERG is the only entry in Soto with multiple isoform matches, none of which are canonical
# There is a second entry in Soto which only matches P11308-3, so will update the first entry to be consistent
Soto_output.at[72, "uniprotID"] = "P11308-3"

In [191]:
# All GSL entries w/ multip matches have at least one canonical match
GSL_output["num_matches"] = [len(_) for _ in GSL_output["matching_isoforms"]]
GSL_output[GSL_output["num_matches"] > 1]

,Unnamed: 0,GeneName,Start,End,uniprotID,Matching Isoforms,orig_uniprotID,Reference,Sequence,Length,TileType,matching_isoforms,num_matches
5,5,TP73,1,46,O15350,"[['O15350', 'O15350-13', 'O15350-2', 'O15350-3...",O15350,"activation_regions.txt, GSL (O15350, 1_46)",MAQSTATSPDGGTTFEHLWSSLEPDSTYFDLPQSSRGNNEVVGGTD,46,TF,"[O15350, O15350-13, O15350-2, O15350-3, O15350...",7
9,9,CREB3,1,92,O43889,"[['O43889', 'O43889-3']]",O43889,"activation_regions.txt, GSL (O43889, 1_92)",MELELDAGDQDLLAFLLEESGDLGTAPDEAVRAPLDWALPLSEVPS...,92,TF,"[O43889, O43889-3]",2
13,13,NFATC1,126,218,O95644,"[['O95644', 'O95644-10', 'O95644-2', 'O95644-4...",O95644 / O95644,"TAD_regions.txt / activation_regions.txt, GSL ...",LGLYHNNNQFFHDVEVEDVLPSSKRSPSTATLSLPSLEAYRDPSCL...,93,TF,"[O95644, O95644-10, O95644-2, O95644-4]",4
14,14,ESRRB,203,433,O95718,"[['O95718', 'O95718-1']]",O95718,"transcriptionalactivity_regions.txt, GSL (O957...",PPAKKPLTKIVSYLLVAEPDKLYAMPPPGMPEGDIKALTTLCDLAD...,231,TF,"[O95718, O95718-1]",2
18,18,ESR1,1,184,P03372,"[['P03372', 'P03372-2']]",P03372,"activation_regions.txt, GSL (P03372, 1_184)",MTMTLHTKASGMALLHQIQGNELEPLNRPQLKIPLERPLGEVYLDS...,184,TF,"[P03372, P03372-2]",2
20,20,NR3C1,98,115,P04150,"[['P04150', 'P04150-10', 'P04150-2', 'P04150-3...",P04150,"transcriptionalactivity_regions.txt, GSL (P041...",MGNDLGFPQQGQISLSSG,18,TF,"[P04150, P04150-10, P04150-2, P04150-3, P04150...",7
21,21,GlucocorticoidReceptor,526,556,P04150,"[['P04150', 'P04150-2']]",P04150,"Choi 2000 list, GSL (P04150, 526_556)",PQLTPTLVSLLEVIEPEVLYAGYDSSVPDST,31,TF,"[P04150, P04150-2]",2
22,22,p53 / TP53,1,61,P04637,"[['P04637', 'P04637-2', 'P04637-3'], ['P04637'...",P04637 / P04637 / P04637,"nan / activation_regions.txt / nan, GSL (P0463...",MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...,61,TF,"[P04637, P04637-2, P04637-3]",3
24,24,AP-2,6,117,P05549,"[['P05549', 'P05549']]",Q96CW1,"Choi 2000 list, GSL (P05549, 6_117)",TDNIKYEDCEDRHDGTSNGTARLPQLGTVGQSPYTSAPPLSHTPNA...,112,TF,"[P05549, P05549-2]",2
25,25,AP-2,6,117,P05549,"[['P05549', 'P05549']]",Q96CW1,"Choi 2000 list, GSL (P05549, 6_117)",TDNIKYEDCEDRHDGTSNGTARLPQLGTVGQSPYTSAPPLSHTPNA...,112,TF,"[P05549, P05549-2]",2


In [192]:
active_output["Canonical Transcript ID"] = np.nan
Soto_output["Canonical Transcript ID"] = np.nan
GSL_output["Canonical Transcript ID"] = np.nan

In [193]:
def return_merged_row(uniprotID, df):
    # Only look at rows with the same uniprot ID
    same_uniprotID_rows = df[df["uniprotID"] == uniprotID]
    same_uniprotID_rows = same_uniprotID_rows.sort_values(by = "Start")
    
    # Final dataframe columns
    new_starts = []
    new_ends = []
    genes = []
    AD_names = []
    references = []
    matching_isoforms = []
    transcript_IDs = []
    orig_uniprotIDs = []
    TileTypes = []
    
    # Current row's values
    curr_start = -1
    curr_end = -1
    curr_genes = []
    curr_AD_names = []
    curr_references = []
    curr_matching_isoforms = []
    curr_transcript_IDs = []
    curr_orig_uniprotIDs = []
    curr_TileTypes = []
    
    for i in same_uniprotID_rows.index:
        # Merge current row with next row
        if curr_end >= same_uniprotID_rows.loc[i]["Start"]:
            curr_end = max(curr_end, same_uniprotID_rows.loc[i]["End"])
            curr_genes.append(same_uniprotID_rows.loc[i]["GeneName"])
            curr_references.append(same_uniprotID_rows.loc[i]["Reference"])
            curr_matching_isoforms.append(same_uniprotID_rows.loc[i]["matching_isoforms"])
            curr_transcript_IDs.append(same_uniprotID_rows.loc[i]["Canonical Transcript ID"])
            curr_orig_uniprotIDs.append(same_uniprotID_rows.loc[i]["orig_uniprotID"])
            curr_TileTypes.append(same_uniprotID_rows.loc[i]["TileType"])
        
        # Don't merge current row with next row
        else: 
            new_starts.append(curr_start)
            new_ends.append(curr_end)
            genes.append(" / ".join(set([c.strip() for c in curr_genes])))
            
            curr_AD_names = [str(c) for c in curr_AD_names]
            AD_names.append(" / ".join(curr_AD_names))
            
            curr_references = [str(c) for c in curr_references]
            references.append(" // ".join(curr_references))
            
            # curr_matching_isoforms = [c for c in curr_matching_isoforms]
            matching_isoforms.append(curr_matching_isoforms)
            
            curr_transcript_IDs = [str(c) for c in curr_transcript_IDs]
            transcript_IDs.append(" / ".join(curr_transcript_IDs))
            
            curr_orig_uniprotIDs = [str(c) for c in curr_orig_uniprotIDs]
            orig_uniprotIDs.append(" / ".join(curr_orig_uniprotIDs))
            
            curr_TileTypes = [str(c) for c in curr_TileTypes]
            TileTypes.append(" / ".join(curr_TileTypes))
            
            curr_start = same_uniprotID_rows.loc[i]["Start"]
            curr_end = same_uniprotID_rows.loc[i]["End"]
            
            curr_genes = [same_uniprotID_rows.loc[i]["GeneName"]]
            curr_references = [same_uniprotID_rows.loc[i]["Reference"]]
            curr_matching_isoforms = [same_uniprotID_rows.loc[i]["matching_isoforms"]]
            curr_transcript_IDs = [same_uniprotID_rows.loc[i]["Canonical Transcript ID"]]
            curr_orig_uniprotIDs = [same_uniprotID_rows.loc[i]["orig_uniprotID"]]
            curr_TileTypes = [same_uniprotID_rows.loc[i]["TileType"]]
    
    # Append the last values
    new_starts.append(curr_start)
    new_ends.append(curr_end)
    
    genes.append(" / ".join(set([c.strip() for c in curr_genes])))
    
    curr_AD_names = [str(c) for c in curr_AD_names]
    AD_names.append(" / ".join(curr_AD_names))
    
    curr_references = [str(c) for c in curr_references]
    references.append(" // ".join(curr_references))
    
    # curr_matching_isoforms = [c for c in curr_matching_isoforms]
    matching_isoforms.append(curr_matching_isoforms)
            
    curr_transcript_IDs = [str(c) for c in curr_transcript_IDs]
    transcript_IDs.append(" / ".join(curr_transcript_IDs))
            
    curr_orig_uniprotIDs = [str(c) for c in curr_orig_uniprotIDs]
    orig_uniprotIDs.append(" / ".join(curr_orig_uniprotIDs))
    
    curr_TileTypes = [str(c) for c in curr_TileTypes]
    TileTypes.append(" / ".join(curr_TileTypes))
    
    # Remove the first (because it is just -1 or "")
    new_starts = new_starts[1:]
    new_ends = new_ends[1:]
    genes = genes[1:]
    references = references[1:]
    matching_isoforms = matching_isoforms[1:]
    transcript_IDs = transcript_IDs[1:]
    orig_uniprotIDs = orig_uniprotIDs[1:]
    TileTypes = TileTypes[1:]
    
    return pd.DataFrame({"Gene": genes,
                         "Start": new_starts,
                        "End": new_ends,
                        "uniprotID": uniprotID,
                         "Matching Isoforms" : matching_isoforms, 
                         "Canonical Transcript ID" : transcript_IDs,
                         "orig_uniprotID" :orig_uniprotIDs,
                         "Reference": references,
                         "TileType" : TileTypes
                        })

In [194]:
mapped_known_ADs = pd.concat([Stanford_output, active_output, Soto_output, GSL_output])

In [195]:
matches_found = 0
mapped_known_ADs = mapped_known_ADs.reset_index(drop = True)

for i in mapped_known_ADs.index:
    uniprotID = mapped_known_ADs["uniprotID"].iloc[i]
    orig_start = mapped_known_ADs["Start"].iloc[i]
    orig_end = mapped_known_ADs["End"].iloc[i]

    canonical_coords = return_canonical_coords(uniprotID, mapped_known_ADs)

    if canonical_coords:
        print("Changing: " + mapped_known_ADs["GeneName"].iloc[i])
        print("\tFrom: " + str((uniprotID, orig_start, orig_end)))
        print("\tTo:" + str(canonical_coords))
        print()
        canonical_isoform_id, start, end = canonical_coords
        mapped_known_ADs.at[i, 'uniprotID'] = canonical_isoform_id
        mapped_known_ADs.at[i, 'Start'] = start
        mapped_known_ADs.at[i, 'End'] = end
        matches_found += 1

print(str(matches_found) + " matches found")

Changing: ARNTL
	From: ('O00327-8', np.float64(542.0), np.float64(625.0))
	To:('O00327', 543, 626)

Changing: EBF3
	From: ('Q9H4W6-2', np.float64(272.0), np.float64(351.0))
	To:('Q9H4W6', 281, 360)

Changing: EP400
	From: ('Q96L91-2', np.float64(2061.0), np.float64(2140.0))
	To:('Q96L91', 2097, 2176)

Changing: FOXM1
	From: ('Q08050-3', np.float64(722.0), np.float64(801.0))
	To:('Q08050', 684, 763)

Changing: MEN1
	From: ('O00255-2', np.float64(341.0), np.float64(420.0))
	To:('O00255', 346, 425)

No canonical match for O75030-2
	Canonical full protein sequence:
MQSESGIVPDFEVGEEFHEEPKTYYELKSQPLKSSSSAEHPGASKPPISSSSMTSRILLRQQLMREQMQEQERREQQQKLQAAQFMQQRVPVSQTPAINVSVPTTLPSATQVPMEVLKVQTHLENPTKYHIQQAQRQQVKQYLSTTLANKHANQVLSLPCPNQPGDHVMPPVPGSSAPNSPMAMLTLNSNCEKEGFYKFEEQNRAESECPGMNTHSRASCMQMDDVIDDIISLESSYNEEILGLMDPALQMANTLPVSGNLIDLYGNQGLPPPGLTISNSCPANLPNIKRELTACIFPTESEARALAKERQKKDNHNLIERRRRFNINDRIKELGTLIPKSNDPDMRWNKGTILKASVDYIRKLQREQQRAKELENRQKKLEHANRHLLLRIQELEMQARAHGLSLIPSTGLCSPDLVNRIIKQEPVLENCS

In [196]:
mapped_known_ADs[mapped_known_ADs["GeneName"].str.contains("MYT1L")]

,GeneName,Start,End,uniprotID,Reference,Sequence,orig_uniprotID,Canonical Transcript ID,TileType,matching_isoforms,num_matches,first_match,first_match_not_canonical,Unnamed: 0,Matching Isoforms,Length
329,MYT1L,152.0,251.0,Q9UL68,"DelRosso et al. (Q9UL68-4, 152_251)",EEEEEEEEEEEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNY...,Q9UL68,ENST00000399161,TF,[Q9UL68-4],NaN,NaN,NaN,NaN,NaN,NaN
330,MYT1L,192.0,271.0,Q9UL68,"DelRosso et al. (Q9UL68-4, 192_271)",DEYDNYDELVAKSLLNLGKIAEDAAYRARTESEMNSNTSNSLEDDS...,Q9UL68,ENST00000399161,TF,[Q9UL68-4],NaN,NaN,NaN,NaN,NaN,NaN
331,MYT1L,312.0,391.0,Q9UL68,"DelRosso et al. (Q9UL68-4, 312_391)",EKMVEESDEEVCLSSLECLRNQCFDLARKLSETNPQERNPQQNMNI...,Q9UL68,ENST00000399161,TF,[Q9UL68-4],NaN,NaN,NaN,NaN,NaN,NaN
684,MYT1L,214.0,425.0,Q9UL68,"PMID: 29291346, Soto (Q9UL68, 214_425)",DAAYRARTESEMNSNTSNSLEDDSDKNENLGRKSELSLDLDSDVVR...,Q9UL68,NaN,TF,"[Q9UL68, Q9UL68-4]",2.0,Q9UL68,False,NaN,NaN,NaN


In [197]:
mapped_known_ADs[mapped_known_ADs["uniprotID"] =="Q9UL68"]

,GeneName,Start,End,uniprotID,Reference,Sequence,orig_uniprotID,Canonical Transcript ID,TileType,matching_isoforms,num_matches,first_match,first_match_not_canonical,Unnamed: 0,Matching Isoforms,Length
329,MYT1L,152.0,251.0,Q9UL68,"DelRosso et al. (Q9UL68-4, 152_251)",EEEEEEEEEEEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNY...,Q9UL68,ENST00000399161,TF,[Q9UL68-4],NaN,NaN,NaN,NaN,NaN,NaN
330,MYT1L,192.0,271.0,Q9UL68,"DelRosso et al. (Q9UL68-4, 192_271)",DEYDNYDELVAKSLLNLGKIAEDAAYRARTESEMNSNTSNSLEDDS...,Q9UL68,ENST00000399161,TF,[Q9UL68-4],NaN,NaN,NaN,NaN,NaN,NaN
331,MYT1L,312.0,391.0,Q9UL68,"DelRosso et al. (Q9UL68-4, 312_391)",EKMVEESDEEVCLSSLECLRNQCFDLARKLSETNPQERNPQQNMNI...,Q9UL68,ENST00000399161,TF,[Q9UL68-4],NaN,NaN,NaN,NaN,NaN,NaN
684,MYT1L,214.0,425.0,Q9UL68,"PMID: 29291346, Soto (Q9UL68, 214_425)",DAAYRARTESEMNSNTSNSLEDDSDKNENLGRKSELSLDLDSDVVR...,Q9UL68,NaN,TF,"[Q9UL68, Q9UL68-4]",2.0,Q9UL68,False,NaN,NaN,NaN


In [198]:
# mapped_known_ADs.reset_index(inplace= True, drop=True)

dfs = []
i = 0
for uniprotID in mapped_known_ADs["uniprotID"].unique():
    dfs.append(return_merged_row(uniprotID, mapped_known_ADs))

merged_mapped_known_ADs = pd.concat(dfs)
merged_mapped_known_ADs = merged_mapped_known_ADs.reset_index(drop = True)
merged_mapped_known_ADs

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType
0,ABRAXAS1,121.0,200.0,Q6UWZ7,[[Q6UWZ7]],ENST00000321945,Q6UWZ7,"DelRosso et al. (Q6UWZ7, 121_200)",CR
1,AHR,118.0,126.0,P35869,[[P35869]],nan,P35869,"transcriptionalactivity_regions.txt, GSL (P358...",TF
2,AHR,266.0,268.0,P35869,[[P35869]],nan,P35869,"transcriptionalactivity_regions.txt, GSL (P358...",TF
3,AHR,532.0,848.0,P35869,"[[P35869], [P35869], [P35869]]",ENST00000242057 / nan / ENST00000242057,P35869 / P35869 / P35869,"DelRosso et al. (P35869, 532_641) // PMID: 879...",TF / TF / TF
4,AKAP8,2.0,81.0,O43823,[[O43823]],ENST00000269701,O43823,"DelRosso et al. (O43823, 2_81)",TF
...,...,...,...,...,...,...,...,...,...
739,HIVEP3,211.0,1074.0,Q5T1R4,"[[Q5T1R4, Q5T1R4-2]]",nan,Q5T1R4,"activation_regions.txt, GSL (Q5T1R4, 211_1074)",TF
740,ZNF516,1.0,431.0,Q92618,[[Q92618]],nan,Q92618,"activation_regions.txt, GSL (Q92618, 1_431)",TF
741,CITE1/MSG1,145.0,193.0,Q99966,[[Q99966]],nan,Q99966 / Q99966,Mikko Taipali AND Choi 2000 list / Mikko Taipa...,TF
742,CITED2,220.0,269.0,Q99967,[[Q99967]],nan,Q99967 / Q99967,"MVS fragment / Berlow et al, GSL (Q99967, 220_...",TF


In [199]:
# Now, adding sequences using isoform_df
merged = pd.merge(merged_mapped_known_ADs, isoform_df, on = "uniprotID", how = "left")
merged

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,id,seq
0,ABRAXAS1,121.0,200.0,Q6UWZ7,[[Q6UWZ7]],ENST00000321945,Q6UWZ7,"DelRosso et al. (Q6UWZ7, 121_200)",CR,sp|Q6UWZ7|ABRX1_HUMAN BRCA1-A complex subunit ...,MEGESTSAVLSGFVLGALAFQHLNTDSDTEGFLLGEVKGEAKNSIT...
1,AHR,118.0,126.0,P35869,[[P35869]],nan,P35869,"transcriptionalactivity_regions.txt, GSL (P358...",TF,sp|P35869|AHR_HUMAN Aryl hydrocarbon receptor ...,MNSSSANITYASRKRRKPVQKTVKPIPAEGIKSNPSKRHRDRLNTE...
2,AHR,266.0,268.0,P35869,[[P35869]],nan,P35869,"transcriptionalactivity_regions.txt, GSL (P358...",TF,sp|P35869|AHR_HUMAN Aryl hydrocarbon receptor ...,MNSSSANITYASRKRRKPVQKTVKPIPAEGIKSNPSKRHRDRLNTE...
3,AHR,532.0,848.0,P35869,"[[P35869], [P35869], [P35869]]",ENST00000242057 / nan / ENST00000242057,P35869 / P35869 / P35869,"DelRosso et al. (P35869, 532_641) // PMID: 879...",TF / TF / TF,sp|P35869|AHR_HUMAN Aryl hydrocarbon receptor ...,MNSSSANITYASRKRRKPVQKTVKPIPAEGIKSNPSKRHRDRLNTE...
4,AKAP8,2.0,81.0,O43823,[[O43823]],ENST00000269701,O43823,"DelRosso et al. (O43823, 2_81)",TF,sp|O43823|AKAP8_HUMAN A-kinase anchor protein ...,MDQGYGGYGAWSAGPANTQGAYGTGVASWQGYENYNYYGAQNTSVT...
...,...,...,...,...,...,...,...,...,...,...,...
739,HIVEP3,211.0,1074.0,Q5T1R4,"[[Q5T1R4, Q5T1R4-2]]",nan,Q5T1R4,"activation_regions.txt, GSL (Q5T1R4, 211_1074)",TF,sp|Q5T1R4|ZEP3_HUMAN Transcription factor HIVE...,MDPEQSVKGTKKAEGSPRKRLTKGEAIQTSVSSSVPYPGSGTAATQ...
740,ZNF516,1.0,431.0,Q92618,[[Q92618]],nan,Q92618,"activation_regions.txt, GSL (Q92618, 1_431)",TF,sp|Q92618|ZN516_HUMAN Zinc finger protein 516 ...,MDRNREAEMELRRGPSPTRAGRGHEVDGDKATCHTCCICGKSFPFQ...
741,CITE1/MSG1,145.0,193.0,Q99966,[[Q99966]],nan,Q99966 / Q99966,Mikko Taipali AND Choi 2000 list / Mikko Taipa...,TF,sp|Q99966|CITE1_HUMAN Cbp/p300-interacting tra...,MPTTSRPALDVKGGTSPAKEDANQEMSSVAYSNLAVKDRKAVAILH...
742,CITED2,220.0,269.0,Q99967,[[Q99967]],nan,Q99967 / Q99967,"MVS fragment / Berlow et al, GSL (Q99967, 220_...",TF,sp|Q99967|CITE2_HUMAN Cbp/p300-interacting tra...,MADHMMAMNHGRFPDGTNGLHHHPAHRMGMGQFPSPHHHQQQQPQH...


In [200]:
merged["Start"] = merged["Start"].astype(int)
merged["End"] = merged["End"].astype(int)

merged["ProteinRegionSeq"] = [seq[start - 1: end] for seq, start, end in zip(merged["seq"],
                                                                     merged["Start"],
                                                                     merged["End"])]
merged = merged.drop(columns = {"seq", "id"})
merged

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq
0,ABRAXAS1,121,200,Q6UWZ7,[[Q6UWZ7]],ENST00000321945,Q6UWZ7,"DelRosso et al. (Q6UWZ7, 121_200)",CR,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...
1,AHR,118,126,P35869,[[P35869]],nan,P35869,"transcriptionalactivity_regions.txt, GSL (P358...",TF,LLQALNGFV
2,AHR,266,268,P35869,[[P35869]],nan,P35869,"transcriptionalactivity_regions.txt, GSL (P358...",TF,FAI
3,AHR,532,848,P35869,"[[P35869], [P35869], [P35869]]",ENST00000242057 / nan / ENST00000242057,P35869 / P35869 / P35869,"DelRosso et al. (P35869, 532_641) // PMID: 879...",TF / TF / TF,QDSKNSDLYSIMKNLGIDFEDIRHMQNEKFFRNDFSGEVDFRDIDL...
4,AKAP8,2,81,O43823,[[O43823]],ENST00000269701,O43823,"DelRosso et al. (O43823, 2_81)",TF,DQGYGGYGAWSAGPANTQGAYGTGVASWQGYENYNYYGAQNTSVTT...
...,...,...,...,...,...,...,...,...,...,...
739,HIVEP3,211,1074,Q5T1R4,"[[Q5T1R4, Q5T1R4-2]]",nan,Q5T1R4,"activation_regions.txt, GSL (Q5T1R4, 211_1074)",TF,IRSHTGERPYPCGPCGFSFKTKSNLYKHRKSHAHRIKAGLASGMGG...
740,ZNF516,1,431,Q92618,[[Q92618]],nan,Q92618,"activation_regions.txt, GSL (Q92618, 1_431)",TF,MDRNREAEMELRRGPSPTRAGRGHEVDGDKATCHTCCICGKSFPFQ...
741,CITE1/MSG1,145,193,Q99966,[[Q99966]],nan,Q99966 / Q99966,Mikko Taipali AND Choi 2000 list / Mikko Taipa...,TF,PAIIDSDPVDEEVLMSLVVELGLDRANELPELWLGQNEFDFTADFPSSC
742,CITED2,220,269,Q99967,[[Q99967]],nan,Q99967 / Q99967,"MVS fragment / Berlow et al, GSL (Q99967, 220_...",TF,TDFIDEEVLMSLVIEMGLDRIKELPELWLGQNEFDFMTDFVCKQQP...


In [201]:
merged = merged.sort_values(by = "Gene")
merged = merged.reset_index(drop = True)

In [202]:
# Starting a notes column

merged["Notes"] = ""

In [203]:
merged[merged["orig_uniprotID"] == "P86452"]

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
701,ZBED6,731,810,O75152,[[O75152]],ENST00000545588,P86452,"DelRosso et al. (O75152, 731_810)",TF,TQSSSDSSPPEVSGPSSSQMSMKTRRLSSASTGKPPLSVEDDFEKL...,


In [204]:
merged.at[702, "Notes"] = "The ENST ID 'ENST00000545588' used in DelRosso et al. is retired. The domain sequence actually matches UniProtID O75152, instead of the original UniProtID P86452 given. The Gene name given here matches that corresponding to the ENST ID."

In [205]:
# Overlap cases
merged["ProteinRegionSeq"].value_counts()

ProteinRegionSeq
QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAALLDTALVNSGILTIDVASVSSTLAGHLPANNNNSVGQAVDPPSLMATSDPPQSLDTSLFFGTAATGFQQSSLNMDEVSSVSV                                    2
AGNGGPPEGSLDVLQSWCEKLAEIIWQNRQQIRRAEHLCQQLPIPGPVEEMLAEVNATITDIISALVTSTFIIEKQPPQV                                                                                    2
LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPLVVANLGMSEQLGYKTVSGSCMSTGFSRAVQTHSS                                                                                    1
MAGNDCGALLDEELSSFFLNYLADTQGGGSGEEQLYADFPELDLSQLDASDFDSATCFGELQWCPENSETEPNQYSPDDSELFQIDSENEALLAELTKTLDDIPEDDVGLAAFPALDGGDALSCTSASPAPSSAPPSPAPEKPSAPAPEVDELSLLQKLL    1
CLPTPPSNIFGGLDESLLARAEALAAVDIVSQSKSHHHHPPHHSPFKPDATYHTMNTIPCTSAASSSSVPISHPSALAGTHHHHHHHHHHHHQPHQALEGELLEHLSPGLALGAMAGPDGAVVSTPAHAPHMATMNPMHQAALSMAH                 1
                                                                                                                                                         

In [206]:
merged[merged["ProteinRegionSeq"] == "QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAALLDTALVNSGILTIDVASVSSTLAGHLPANNNNSVGQAVDPPSLMATSDPPQSLDTSLFFGTAATGFQQSSLNMDEVSSVSV"]

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
738,ZXDA,572,699,P98168,"[[P98168], [P98168]]",nan / nan,P98168 / P98168 / P98168,"PMID: 17493635, Soto (P98168, 572_698) // R4TA...",TF / TF,QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...,
739,ZXDB,576,703,P98169,[[P98169]],nan,P98169 / P98169,"R4TA_regions.txt / activation_regions.txt, GSL...",TF,QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...,


In [207]:
merged[merged["ProteinRegionSeq"] == "EAHLKPEHHYAFNHPFSINNLMSSEQQHHHSHHHHQPHKMDLKAYEQVMHYPGYGSPMPGSLAMGPVTNKTGLDASPLAADTSYYQGVYSRPIMNSS"]

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
166,FOXA2/HNF3B / FOXA2,361,457,Q9Y261,"[[Q9Y261], [Q9Y261], [Q9Y261-2]]",nan / nan / nan,Q9Y261 / Q9Y261 / Q9Y261,"PMID: 1324404, Soto (Q9Y261, 361_457) // activ...",TF / TF / TF,EAHLKPEHHYAFNHPFSINNLMSSEQQHHHSHHHHQPHKMDLKAYE...,


In [208]:
merged["Reference"].iloc[167]

'PMID: 11782474, Soto (Q12948, 435_553) // R4TA_regions.txt / activation_regions.txt, GSL (Q12948, 466_553)'

In [209]:
# # Merge the two FOXA2 entries since the sequences are identical
# merged.at[167, "Matching Isoforms"] = [["Q9Y261"], ["Q9Y261"], ["Q9Y261"]]
# merged.at[167, "Canonical Transcript ID"] = "nan / nan / nan"
# merged.at[167, "orig_uniprotID"] = "Q9Y261 / Q9Y261 / Q9Y261"
# merged.at[167, "Reference"] = "PMID: 1324404, Soto / activation_regions.txt, GSL / Choi 2000 list, GSL"
# merged.at[167, "Notes"] = "The coordinates and domain sequence originally given by the Choi 2000 list (367-463) actually match isoform Q9Y261-2. However, the sequence of that domain is the same as the sequence given by the Soto and other GSL entry for the canonical isoform. Therefore, these entries have been merged to reflect the coordinates for the canonical isoform."
# merged = merged.drop(168)

In [210]:
merged[merged["ProteinRegionSeq"] == "AGNGGPPEGSLDVLQSWCEKLAEIIWQNRQQIRRAEHLCQQLPIPGPVEEMLAEVNATITDIISALVTSTFIIEKQPPQV"]

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
608,STAT5A,262,341,P42229,[[P42229]],ENST00000345506,P42229,"DelRosso et al. (P42229, 262_341)",TF,AGNGGPPEGSLDVLQSWCEKLAEIIWQNRQQIRRAEHLCQQLPIPG...,
610,STAT5B,262,341,P51692,[[P51692]],ENST00000293328,P51692,"DelRosso et al. (P51692, 262_341)",TF,AGNGGPPEGSLDVLQSWCEKLAEIIWQNRQQIRRAEHLCQQLPIPG...,


In [211]:
merged = merged.reset_index(drop = True)

In [212]:
len(merged) # There should be 760

744

In [213]:
# Adding whether each entry is for a TF
merged

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
0,ABRAXAS1,121,200,Q6UWZ7,[[Q6UWZ7]],ENST00000321945,Q6UWZ7,"DelRosso et al. (Q6UWZ7, 121_200)",CR,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,
1,AEBP1,1088,1158,Q8IUX7,[[Q8IUX7]],nan,Q8IUX7,"Staller Activity Data (Q8IUX7, 1087.0_1158.0)",TF,EVVTEFGTEVEPEFGTKVEPEFETQLEPEFETQLEPEFEEEEEEEK...,
2,AHCTF1,1445,1698,Q8WYP5,"[[Q8WYP5], [Q8WYP5]]",nan / nan,Q8WYP5 / Q8WYP5,"PMID: 11952839, Soto (Q8WYP5, 1445_1698) // tr...",TF / TF,IRANDNKSMADVLGDGGNSSLTISEGPIVSERRLNQEVALNLKEDH...,
3,AHR,118,126,P35869,[[P35869]],nan,P35869,"transcriptionalactivity_regions.txt, GSL (P358...",TF,LLQALNGFV,
4,AHR,266,268,P35869,[[P35869]],nan,P35869,"transcriptionalactivity_regions.txt, GSL (P358...",TF,FAI,
...,...,...,...,...,...,...,...,...,...,...,...
739,ZXDB,576,703,P98169,[[P98169]],nan,P98169 / P98169,"R4TA_regions.txt / activation_regions.txt, GSL...",TF,QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...,
740,ZXDC,579,688,Q2QGD7,"[[Q2QGD7, Q2QGD7-2], [Q2QGD7, Q2QGD7-2]]",nan / nan,Q2QGD7 / Q2QGD7 / Q2QGD7,"PMID: 16600381, Soto (Q2QGD7, 579_688) // R4TA...",TF / TF,DSPLVLGTAATVLQQGSFSVDDVQTVSAGALGCLVALPMKNLSDDP...,
741,c-Jun/AP-1,238,257,P05412,[[P05412]],nan,P05412,"Choi 2000 list, GSL (P05412, 238_257)",TF,ETPPLSPIDMESQERIKAER,
742,c-Myb / MYB / MYB,232,361,P10242,"[[P10242, P10242-10, P10242-4, P10242-5, P1024...",ENST00000367814 / nan / nan,P10242 / P10242 / P10242 / P10242 / P10242,"DelRosso et al. (P10242, 232_361) // PMID: 941...",TF / TF / TF,LPATGQPTVNNDYSYYHISEAQNVSSHVPYPVALHVNIVNVPQPAA...,


In [214]:
merged["TileType"].value_counts()

TileType
TF                                  419
TF / TF                             133
CR                                   89
TF / TF / TF                         48
TF / TF / TF / TF                    31
CR / CR                              14
TF / TF / TF / TF / TF                6
TF / TF / TF / TF / TF / TF / TF      2
TF / TF / TF / TF / TF / TF           1
CR / CR / CR                          1
Name: count, dtype: int64

In [215]:
# All labels are the same
merged["TileType"] = merged["TileType"].str.split(" / ").str[0]
merged

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
0,ABRAXAS1,121,200,Q6UWZ7,[[Q6UWZ7]],ENST00000321945,Q6UWZ7,"DelRosso et al. (Q6UWZ7, 121_200)",CR,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,
1,AEBP1,1088,1158,Q8IUX7,[[Q8IUX7]],nan,Q8IUX7,"Staller Activity Data (Q8IUX7, 1087.0_1158.0)",TF,EVVTEFGTEVEPEFGTKVEPEFETQLEPEFETQLEPEFEEEEEEEK...,
2,AHCTF1,1445,1698,Q8WYP5,"[[Q8WYP5], [Q8WYP5]]",nan / nan,Q8WYP5 / Q8WYP5,"PMID: 11952839, Soto (Q8WYP5, 1445_1698) // tr...",TF,IRANDNKSMADVLGDGGNSSLTISEGPIVSERRLNQEVALNLKEDH...,
3,AHR,118,126,P35869,[[P35869]],nan,P35869,"transcriptionalactivity_regions.txt, GSL (P358...",TF,LLQALNGFV,
4,AHR,266,268,P35869,[[P35869]],nan,P35869,"transcriptionalactivity_regions.txt, GSL (P358...",TF,FAI,
...,...,...,...,...,...,...,...,...,...,...,...
739,ZXDB,576,703,P98169,[[P98169]],nan,P98169 / P98169,"R4TA_regions.txt / activation_regions.txt, GSL...",TF,QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...,
740,ZXDC,579,688,Q2QGD7,"[[Q2QGD7, Q2QGD7-2], [Q2QGD7, Q2QGD7-2]]",nan / nan,Q2QGD7 / Q2QGD7 / Q2QGD7,"PMID: 16600381, Soto (Q2QGD7, 579_688) // R4TA...",TF,DSPLVLGTAATVLQQGSFSVDDVQTVSAGALGCLVALPMKNLSDDP...,
741,c-Jun/AP-1,238,257,P05412,[[P05412]],nan,P05412,"Choi 2000 list, GSL (P05412, 238_257)",TF,ETPPLSPIDMESQERIKAER,
742,c-Myb / MYB / MYB,232,361,P10242,"[[P10242, P10242-10, P10242-4, P10242-5, P1024...",ENST00000367814 / nan / nan,P10242 / P10242 / P10242 / P10242 / P10242,"DelRosso et al. (P10242, 232_361) // PMID: 941...",TF,LPATGQPTVNNDYSYYHISEAQNVSSHVPYPVALHVNIVNVPQPAA...,


In [216]:
# merged.to_csv("../output/known_ADs_considering_isoforms_and_canonical.csv", index = False)

In [217]:
merged[merged["Gene"].str.contains("MYT1L")]

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
366,MYT1L,152,425,Q9UL68,"[[Q9UL68-4], [Q9UL68-4], [Q9UL68, Q9UL68-4], [...",ENST00000399161 / ENST00000399161 / nan / ENST...,Q9UL68 / Q9UL68 / Q9UL68 / Q9UL68,"DelRosso et al. (Q9UL68-4, 152_251) // DelRoss...",TF,EEEEEEEEEEEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNY...,


In [218]:
merged 

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
0,ABRAXAS1,121,200,Q6UWZ7,[[Q6UWZ7]],ENST00000321945,Q6UWZ7,"DelRosso et al. (Q6UWZ7, 121_200)",CR,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,
1,AEBP1,1088,1158,Q8IUX7,[[Q8IUX7]],nan,Q8IUX7,"Staller Activity Data (Q8IUX7, 1087.0_1158.0)",TF,EVVTEFGTEVEPEFGTKVEPEFETQLEPEFETQLEPEFEEEEEEEK...,
2,AHCTF1,1445,1698,Q8WYP5,"[[Q8WYP5], [Q8WYP5]]",nan / nan,Q8WYP5 / Q8WYP5,"PMID: 11952839, Soto (Q8WYP5, 1445_1698) // tr...",TF,IRANDNKSMADVLGDGGNSSLTISEGPIVSERRLNQEVALNLKEDH...,
3,AHR,118,126,P35869,[[P35869]],nan,P35869,"transcriptionalactivity_regions.txt, GSL (P358...",TF,LLQALNGFV,
4,AHR,266,268,P35869,[[P35869]],nan,P35869,"transcriptionalactivity_regions.txt, GSL (P358...",TF,FAI,
...,...,...,...,...,...,...,...,...,...,...,...
739,ZXDB,576,703,P98169,[[P98169]],nan,P98169 / P98169,"R4TA_regions.txt / activation_regions.txt, GSL...",TF,QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...,
740,ZXDC,579,688,Q2QGD7,"[[Q2QGD7, Q2QGD7-2], [Q2QGD7, Q2QGD7-2]]",nan / nan,Q2QGD7 / Q2QGD7 / Q2QGD7,"PMID: 16600381, Soto (Q2QGD7, 579_688) // R4TA...",TF,DSPLVLGTAATVLQQGSFSVDDVQTVSAGALGCLVALPMKNLSDDP...,
741,c-Jun/AP-1,238,257,P05412,[[P05412]],nan,P05412,"Choi 2000 list, GSL (P05412, 238_257)",TF,ETPPLSPIDMESQERIKAER,
742,c-Myb / MYB / MYB,232,361,P10242,"[[P10242, P10242-10, P10242-4, P10242-5, P1024...",ENST00000367814 / nan / nan,P10242 / P10242 / P10242 / P10242 / P10242,"DelRosso et al. (P10242, 232_361) // PMID: 941...",TF,LPATGQPTVNNDYSYYHISEAQNVSSHVPYPVALHVNIVNVPQPAA...,


In [219]:
orig_merged = pd.read_csv("../output/known_ADs_considering_isoforms_and_canonical.csv")
orig_merged

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
0,ABRAXAS1,121,200,Q6UWZ7,[['Q6UWZ7']],ENST00000321945,Q6UWZ7,DelRosso et al.,CR,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,NaN
1,AEBP1,1088,1158,Q8IUX7,[['Q8IUX7']],NaN,Q8IUX7,Staller Activity Data,TF,EVVTEFGTEVEPEFGTKVEPEFETQLEPEFETQLEPEFEEEEEEEK...,NaN
2,AHCTF1,1445,1698,Q8WYP5,"[['Q8WYP5'], ['Q8WYP5']]",nan / nan,Q8WYP5 / Q8WYP5,"PMID: 11952839, Soto / transcriptionalactivity...",TF,IRANDNKSMADVLGDGGNSSLTISEGPIVSERRLNQEVALNLKEDH...,NaN
3,AHR,118,126,P35869,[['P35869']],NaN,P35869,"transcriptionalactivity_regions.txt, GSL",TF,LLQALNGFV,NaN
4,AHR,266,268,P35869,[['P35869']],NaN,P35869,"transcriptionalactivity_regions.txt, GSL",TF,FAI,NaN
...,...,...,...,...,...,...,...,...,...,...,...
739,ZSCAN20,262,341,P17040,"[['P17040', 'P17040-3']]",ENST00000361328,P17040,DelRosso et al.,TF,PSNTSEKEQGPEFWGLSLINSGKRSTADYSLDNEPAQALTWRDSRA...,NaN
740,ZXDA,572,699,P98168,"[['P98168'], ['P98168']]",nan / nan,P98168 / P98168 / P98168,"PMID: 17493635, Soto / R4TA_regions.txt / acti...",TF,QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...,NaN
741,ZXDB,576,703,P98169,[['P98169']],NaN,P98169 / P98169,"R4TA_regions.txt / activation_regions.txt, GSL",TF,QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...,NaN
742,ZXDC,579,688,Q2QGD7,"[['Q2QGD7', 'Q2QGD7-2'], ['Q2QGD7', 'Q2QGD7-2']]",nan / nan,Q2QGD7 / Q2QGD7 / Q2QGD7,"PMID: 16600381, Soto / R4TA_regions.txt / acti...",TF,DSPLVLGTAATVLQQGSFSVDDVQTVSAGALGCLVALPMKNLSDDP...,NaN


In [220]:
orig_merged.sort_values(by = "Gene")

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
0,ABRAXAS1,121,200,Q6UWZ7,[['Q6UWZ7']],ENST00000321945,Q6UWZ7,DelRosso et al.,CR,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,NaN
1,AEBP1,1088,1158,Q8IUX7,[['Q8IUX7']],NaN,Q8IUX7,Staller Activity Data,TF,EVVTEFGTEVEPEFGTKVEPEFETQLEPEFETQLEPEFEEEEEEEK...,NaN
2,AHCTF1,1445,1698,Q8WYP5,"[['Q8WYP5'], ['Q8WYP5']]",nan / nan,Q8WYP5 / Q8WYP5,"PMID: 11952839, Soto / transcriptionalactivity...",TF,IRANDNKSMADVLGDGGNSSLTISEGPIVSERRLNQEVALNLKEDH...,NaN
3,AHR,118,126,P35869,[['P35869']],NaN,P35869,"transcriptionalactivity_regions.txt, GSL",TF,LLQALNGFV,NaN
4,AHR,266,268,P35869,[['P35869']],NaN,P35869,"transcriptionalactivity_regions.txt, GSL",TF,FAI,NaN
...,...,...,...,...,...,...,...,...,...,...,...
739,ZSCAN20,262,341,P17040,"[['P17040', 'P17040-3']]",ENST00000361328,P17040,DelRosso et al.,TF,PSNTSEKEQGPEFWGLSLINSGKRSTADYSLDNEPAQALTWRDSRA...,NaN
740,ZXDA,572,699,P98168,"[['P98168'], ['P98168']]",nan / nan,P98168 / P98168 / P98168,"PMID: 17493635, Soto / R4TA_regions.txt / acti...",TF,QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...,NaN
741,ZXDB,576,703,P98169,[['P98169']],NaN,P98169 / P98169,"R4TA_regions.txt / activation_regions.txt, GSL",TF,QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...,NaN
742,ZXDC,579,688,Q2QGD7,"[['Q2QGD7', 'Q2QGD7-2'], ['Q2QGD7', 'Q2QGD7-2']]",nan / nan,Q2QGD7 / Q2QGD7 / Q2QGD7,"PMID: 16600381, Soto / R4TA_regions.txt / acti...",TF,DSPLVLGTAATVLQQGSFSVDDVQTVSAGALGCLVALPMKNLSDDP...,NaN


In [221]:
merged.sort_values(by = "Gene")

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
0,ABRAXAS1,121,200,Q6UWZ7,[[Q6UWZ7]],ENST00000321945,Q6UWZ7,"DelRosso et al. (Q6UWZ7, 121_200)",CR,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,
1,AEBP1,1088,1158,Q8IUX7,[[Q8IUX7]],nan,Q8IUX7,"Staller Activity Data (Q8IUX7, 1087.0_1158.0)",TF,EVVTEFGTEVEPEFGTKVEPEFETQLEPEFETQLEPEFEEEEEEEK...,
2,AHCTF1,1445,1698,Q8WYP5,"[[Q8WYP5], [Q8WYP5]]",nan / nan,Q8WYP5 / Q8WYP5,"PMID: 11952839, Soto (Q8WYP5, 1445_1698) // tr...",TF,IRANDNKSMADVLGDGGNSSLTISEGPIVSERRLNQEVALNLKEDH...,
3,AHR,118,126,P35869,[[P35869]],nan,P35869,"transcriptionalactivity_regions.txt, GSL (P358...",TF,LLQALNGFV,
4,AHR,266,268,P35869,[[P35869]],nan,P35869,"transcriptionalactivity_regions.txt, GSL (P358...",TF,FAI,
...,...,...,...,...,...,...,...,...,...,...,...
739,ZXDB,576,703,P98169,[[P98169]],nan,P98169 / P98169,"R4TA_regions.txt / activation_regions.txt, GSL...",TF,QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...,
740,ZXDC,579,688,Q2QGD7,"[[Q2QGD7, Q2QGD7-2], [Q2QGD7, Q2QGD7-2]]",nan / nan,Q2QGD7 / Q2QGD7 / Q2QGD7,"PMID: 16600381, Soto (Q2QGD7, 579_688) // R4TA...",TF,DSPLVLGTAATVLQQGSFSVDDVQTVSAGALGCLVALPMKNLSDDP...,
741,c-Jun/AP-1,238,257,P05412,[[P05412]],nan,P05412,"Choi 2000 list, GSL (P05412, 238_257)",TF,ETPPLSPIDMESQERIKAER,
742,c-Myb / MYB / MYB,232,361,P10242,"[[P10242, P10242-10, P10242-4, P10242-5, P1024...",ENST00000367814 / nan / nan,P10242 / P10242 / P10242 / P10242 / P10242,"DelRosso et al. (P10242, 232_361) // PMID: 941...",TF,LPATGQPTVNNDYSYYHISEAQNVSSHVPYPVALHVNIVNVPQPAA...,


In [222]:
orig_merged[orig_merged["Gene"].str.contains("53")]

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
671,TP53 / p53 / TP53,1,101,P04637,"[['P04637', 'P04637-2', 'P04637-3'], ['P04637'...",nan / nan / ENST00000269305,P04637 / P04637 / P04637 / P04637 / P04637,"PMID: 9266967, Soto / nan / activation_regions...",TF,MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...,NaN
734,ZNF532,2,81,Q9HCE3,[['Q9HCE3']],ENST00000336078,Q9HCE3,DelRosso et al.,TF,TMGDMKTPDFDDLLAAFDIPDMVDPKAAIESGHDDHESHMKQNAHG...,NaN


In [223]:
new_imp_cols = merged[["Start", "End", "uniprotID"]]
new_imp_cols = new_imp_cols.sort_values(by = "uniprotID").reset_index(drop = True)
new_imp_cols

,Start,End,uniprotID
0,111,210,A0A024R0Y4
1,162,241,A0A087WXG3
2,862,951,A0A669KBM4
3,1372,1460,A0A669KBM4
4,142,315,A6NJG6
...,...,...,...
739,231,350,Q9Y6F1
740,371,480,Q9Y6J9
741,621,1424,Q9Y6Q9
742,702,841,Q9Y6Y1


In [224]:
old_imp_cols = orig_merged[["Start", "End", "uniprotID"]]
old_imp_cols = old_imp_cols.sort_values(by = "uniprotID").reset_index(drop = True)
old_imp_cols

,Start,End,uniprotID
0,111,210,A0A024R0Y4
1,162,241,A0A087WXG3
2,1372,1460,A0A669KBM4
3,862,951,A0A669KBM4
4,142,315,A6NJG6
...,...,...,...
739,231,350,Q9Y6F1
740,371,480,Q9Y6J9
741,621,1424,Q9Y6Q9
742,702,841,Q9Y6Y1


In [225]:
pd.merge(new_imp_cols, old_imp_cols, how = "inner")
# Looks the same!

,Start,End,uniprotID
0,111,210,A0A024R0Y4
1,162,241,A0A087WXG3
2,862,951,A0A669KBM4
3,1372,1460,A0A669KBM4
4,142,315,A6NJG6
...,...,...,...
739,231,350,Q9Y6F1
740,371,480,Q9Y6J9
741,621,1424,Q9Y6Q9
742,702,841,Q9Y6Y1


In [226]:
# SK 9/5/24: Changed code above to save the original coordinates in reference column
merged

,Gene,Start,End,uniprotID,Matching Isoforms,Canonical Transcript ID,orig_uniprotID,Reference,TileType,ProteinRegionSeq,Notes
0,ABRAXAS1,121,200,Q6UWZ7,[[Q6UWZ7]],ENST00000321945,Q6UWZ7,"DelRosso et al. (Q6UWZ7, 121_200)",CR,LQEHFSNQDLVFLLLTPSIITESCSTHRLEHSLYKPQKGLFHRVPL...,
1,AEBP1,1088,1158,Q8IUX7,[[Q8IUX7]],nan,Q8IUX7,"Staller Activity Data (Q8IUX7, 1087.0_1158.0)",TF,EVVTEFGTEVEPEFGTKVEPEFETQLEPEFETQLEPEFEEEEEEEK...,
2,AHCTF1,1445,1698,Q8WYP5,"[[Q8WYP5], [Q8WYP5]]",nan / nan,Q8WYP5 / Q8WYP5,"PMID: 11952839, Soto (Q8WYP5, 1445_1698) // tr...",TF,IRANDNKSMADVLGDGGNSSLTISEGPIVSERRLNQEVALNLKEDH...,
3,AHR,118,126,P35869,[[P35869]],nan,P35869,"transcriptionalactivity_regions.txt, GSL (P358...",TF,LLQALNGFV,
4,AHR,266,268,P35869,[[P35869]],nan,P35869,"transcriptionalactivity_regions.txt, GSL (P358...",TF,FAI,
...,...,...,...,...,...,...,...,...,...,...,...
739,ZXDB,576,703,P98169,[[P98169]],nan,P98169 / P98169,"R4TA_regions.txt / activation_regions.txt, GSL...",TF,QDLLAQLEAANSLTPSSELTSQRQNDLSDAEIVSLFSDVPDSTSAA...,
740,ZXDC,579,688,Q2QGD7,"[[Q2QGD7, Q2QGD7-2], [Q2QGD7, Q2QGD7-2]]",nan / nan,Q2QGD7 / Q2QGD7 / Q2QGD7,"PMID: 16600381, Soto (Q2QGD7, 579_688) // R4TA...",TF,DSPLVLGTAATVLQQGSFSVDDVQTVSAGALGCLVALPMKNLSDDP...,
741,c-Jun/AP-1,238,257,P05412,[[P05412]],nan,P05412,"Choi 2000 list, GSL (P05412, 238_257)",TF,ETPPLSPIDMESQERIKAER,
742,c-Myb / MYB / MYB,232,361,P10242,"[[P10242, P10242-10, P10242-4, P10242-5, P1024...",ENST00000367814 / nan / nan,P10242 / P10242 / P10242 / P10242 / P10242,"DelRosso et al. (P10242, 232_361) // PMID: 941...",TF,LPATGQPTVNNDYSYYHISEAQNVSSHVPYPVALHVNIVNVPQPAA...,


In [227]:
merged.to_csv("../output/known_ADs_considering_isoforms_and_canonical_with_ref_coords.csv", index = False)